### Install Packages

In [1]:
%pip install llama-index==0.10.18 llama-index-llms-groq==0.1.3 groq==0.4.2 llama-index-embeddings-huggingface==0.2.0

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


### Import Libraries

In [25]:
from llama_index.core import (
    VectorStoreIndex,
    SimpleDirectoryReader,
    StorageContext,
    ServiceContext,
    load_index_from_storage
)
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core.node_parser import SentenceSplitter
from llama_index.llms.groq import Groq
import os
from dotenv import load_dotenv
load_dotenv()
from sentence_transformers import SentenceTransformer
import warnings
warnings.filterwarnings('ignore')

In [26]:
# from google.colab import userdata
# GROQ_API_KEY = userdata.get('groq')

GROQ_API_KEY = os.getenv("GROQ_API_KEY")

### Data Ingestion

In [27]:
# data ingestion
reader = SimpleDirectoryReader(input_files=["./netprobe.log"])
documents = reader.load_data()

https://docs.llamaindex.ai/en/stable/module_guides/loading/simpledirectoryreader/

In [29]:
# The pdf doc has 86 pages
print(len(documents))
# print(documents)

1


In [30]:
# The 11 page of the doc
documents[0].metadata

{'file_path': 'netprobe.log',
 'file_name': 'netprobe.log',
 'file_size': 127729,
 'creation_date': '2025-03-17',
 'last_modified_date': '2025-03-17'}

### Chunking

In [31]:
text_splitter = SentenceSplitter(chunk_size=1024, chunk_overlap=200)
nodes = text_splitter.get_nodes_from_documents(documents, show_progress=True)

Parsing nodes: 100%|██████████| 1/1 [00:00<00:00,  8.61it/s]


https://docs.llamaindex.ai/en/stable/module_guides/loading/node_parsers/modules/

In [32]:
len(nodes)

56

In [33]:
nodes[0].metadata

{'file_path': 'netprobe.log',
 'file_name': 'netprobe.log',
 'file_size': 127729,
 'creation_date': '2025-03-17',
 'last_modified_date': '2025-03-17'}

https://chunkviz.up.railway.app/

### Embedding Model

In [40]:

model = SentenceTransformer('all-MiniLM-L6-v2')
print(model)

No sentence-transformers model found with name sentence-transformers/all-MiniLM-L6-v2. Creating a new one with MEAN pooling.


ValueError: Unrecognized model in sentence-transformers/all-MiniLM-L6-v2. Should have a `model_type` key in its config.json, or contain one of the following strings in its name: albert, align, altclip, aria, aria_text, audio-spectrogram-transformer, autoformer, aya_vision, bamba, bark, bart, beit, bert, bert-generation, big_bird, bigbird_pegasus, biogpt, bit, blenderbot, blenderbot-small, blip, blip-2, bloom, bridgetower, bros, camembert, canine, chameleon, chinese_clip, chinese_clip_vision_model, clap, clip, clip_text_model, clip_vision_model, clipseg, clvp, code_llama, codegen, cohere, cohere2, colpali, conditional_detr, convbert, convnext, convnextv2, cpmant, ctrl, cvt, dab-detr, dac, data2vec-audio, data2vec-text, data2vec-vision, dbrx, deberta, deberta-v2, decision_transformer, deepseek_v3, deformable_detr, deit, depth_anything, depth_pro, deta, detr, diffllama, dinat, dinov2, dinov2_with_registers, distilbert, donut-swin, dpr, dpt, efficientformer, efficientnet, electra, emu3, en

https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2

https://huggingface.co/spaces/mteb/leaderboard

### Define LLM Model

In [12]:
llm = Groq(model="llama3-70b-8192", api_key=GROQ_API_KEY)

https://console.groq.com/docs/models

https://console.groq.com/keys

### Configure Service Context

In [13]:
service_context = ServiceContext.from_defaults(embed_model=embed_model, llm=llm)

### Create Vector Store Index

In [14]:
vector_index = VectorStoreIndex.from_documents(documents, show_progress=True, service_context=service_context, node_parser=nodes)

Generating embeddings: 100%|██████████| 86/86 [00:02<00:00, 39.63it/s]


https://docs.llamaindex.ai/en/stable/module_guides/indexing/vector_store_index/

#### Persist/Save Index

In [15]:
vector_index.storage_context.persist(persist_dir="./storage_mini")

#### Define Storage Context

In [16]:
storage_context = StorageContext.from_defaults(persist_dir="./storage_mini")

https://docs.llamaindex.ai/en/stable/api_reference/storage/storage_context/

#### Load Index

In [17]:
index = load_index_from_storage(storage_context, service_context=service_context)

### Define Query Engine

In [18]:
query_engine = index.as_query_engine(service_context=service_context)

https://docs.llamaindex.ai/en/stable/module_guides/deploying/query_engine/

#### Feed in user query

https://docs.llamaindex.ai/en/stable/examples/prompts/prompts_rag/#viewingcustomizing-prompts

## Example-1: Query returning result from the queried document

In [26]:
query = "Explain bonds, debts, and healthy lifestyle (General knowledge) and provide the page reference(s) in a dictionary format inside a list. provide also a description field as a concise summary of about 100 words for each"
resp = query_engine.query(query)

In [27]:
print(resp.response)

Here is the answer:

[
    {
        "term": "Bonds",
        "description": "Bonds are long-term liabilities that a company or institution issues to raise capital. They are essentially debt securities that represent a loan made by an investor to the borrower. In the context of a balance sheet, bonds are listed as long-term liabilities, meaning they are due over 12 months.",
        "page_reference": [4]
    },
    {
        "term": "Debts",
        "description": "Debts refer to the amount of money owed by an individual or organization to another party. In the context of a balance sheet, debts are listed as liabilities, which can be either short-term (due within 12 months) or long-term (due over 12 months).",
        "page_reference": [4]
    },
    {
        "term": "Healthy Lifestyle",
        "description": "Not applicable in this context. The provided context information only discusses finance and accounting concepts, and does not mention healthy lifestyle.",
        "page_referen

## Example-2: Query returning result from the LLM general knowledge as it does not exist in the doc

In [28]:
query = "Explain buyside and sellside and provide the page reference in a dictionary format. provide also a description field as a concise summary of about 100 words"
resp = query_engine.query(query)

In [22]:
print(resp.response)

Since the provided context does not mention "buyside" and "sellside", it is not possible to explain these terms or provide a page reference based on the given context.

Here is an empty dictionary as the answer:

{
"description": "No information available in the provided context.",
"page_reference": None
}

Please note that the context only discusses basic finance concepts, such as the balance sheet, assets, liabilities, and equity, but does not mention "buyside" and "sellside".


https://itsjb13.medium.com/building-a-rag-chatbot-using-llamaindex-groq-with-llama3-chainlit-b1709f770f55

https://docs.llamaindex.ai/en/stable/optimizing/production_rag/

In [29]:
from langchain_groq import ChatGroq

In [30]:
load_dotenv()

True

In [ ]:
# Initialize the Groq LLM outside the RAG knowledge context
llm = ChatGroq(model="llama3-70b-8192")

TypeError: __init__() got an unexpected keyword argument 'proxies'